In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download dependencies

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.5 MB/s 


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 7.4 MB/s 


In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 7.6 MB/s 
     |████████████████████████████████| 180 kB 66.4 MB/s 
     |████████████████████████████████| 139 kB 63.4 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=279b009c155075d8ca8eeed1481caf69a1af9b6717f20d2e680249753fc3ea16
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=685f03f7ebe4f581d04440020f7aae675af1a4bcba05156903e149d389d58b2f
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


### Import libraries

In [5]:
import pandas as pd
import numpy as np
import re
import regex
from tqdm import tqdm

from sklearn import preprocessing, metrics
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

import tensorflow_addons as tfa
import tensorflow as tf

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, RNN
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
import warnings
warnings.filterwarnings('ignore')

### Load data

In [9]:
f_path = '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/dataset/crowdflower_text_emotion.csv'

In [10]:
df = pd.read_csv(f_path)

### Data preprocessing and analysis

In [11]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [12]:
# Check emotions labels

labels = df.sentiment.unique()
print(labels)

['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [13]:
# Remove irrelevant columns
df = df.drop(columns=['tweet_id', 'author'])

In [14]:
# Check for null cells
df['sentiment'].isnull().values.any()


False

In [16]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def clean(x, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(x.lower())
    x = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 2]
    return " ".join(x)

df["content_cleaned"] = df.content.apply(lambda x : clean(x, stop)) 

In [17]:
# Check len of each content
df['content_count'] = df['content_cleaned'].str.split(" ").str.len()

In [19]:
df.head()

,sentiment,content,content_cleaned,content_count
0,empty,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin bad habit earlier...,12
1,sadness,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh waitin your call,8
2,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,4
3,enthusiasm,wants to hang out with friends SOON!,wants hang out with friends soon,6
4,neutral,@dannycastillo We want to trade with someone w...,dannycastillo want trade with someone who has ...,12


### Drop unused columns

In [20]:
df = df.drop(columns=['content_count', 'content'])

### LSTM

In [21]:
# Define max vocab size, max sequence length, and embedding dimension
MAX_NB_WORDS = 41277
MAX_SEQUENCE_LENGTH = 256
EMBEDDING_DIM = 128

# Initialize tokenizer and tokenize text
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['content_cleaned'].values)

# Get unique tokens/vocab size
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 41277 unique tokens.


In [22]:
# Convert to sequence
df.reset_index()
X = tokenizer.texts_to_sequences(df['content_cleaned'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Convert to numerical
Y = pd.get_dummies(df['sentiment']).values

print('Shape of data:', X.shape)
print('Shape of label:', Y.shape)

Shape of data: (40000, 256)
Shape of label: (40000, 13)


In [29]:
# Define hyperparameters

epochs = 50
batch_size = 8

In [30]:
# Define lstm cells
cells = [64, 128, 256]

In [31]:
# Function to build model

def build_lstm(cells):
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))

  model.add(LSTM(cells))
  model.add(Dropout(0.3))
  model.add(Dense(13, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  
  return model

In [32]:
# Run on GPU

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [33]:
# 3-Fold
cv = KFold(n_splits=3, random_state=42, shuffle=False)

# For cells
for i in range(len(cells)):
  f = 1

  # For each fold, train
  for train_index, test_index in cv.split(X, Y):

    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # Wandb init
    # wandb.init(project=f'Text_Emotion_LSTM2_Cell_{cells[i]}', name=f'Fold')

    # Clear session
    tf.keras.backend.clear_session()

    # Move here
    opt = tfa.optimizers.AdamW(lr=1e-5, weight_decay=1e-6)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1)

    # LSTM Model
    model = build_lstm(cells[i])
    
    # Training
    print(f"Training {cells[i]} cells, fold {f}...")
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_test, Y_test),
                        callbacks=[reduce_lr, WandbCallback(monitor="val_loss", verbose=0, mode="auto", save_weights_only=False, save_model=False)])

    f = f+1

  # wandb.finish()


accuracy,▁▄▅▅▆▇█
epoch,▁▂▃▅▆▇█
loss,█▃▃▂▂▁▁
lr,▁▁▁▁▁▁▁
val_accuracy,▆▆▅▂▃▁█
val_loss,█▅▄▄▃▃▁
accuracy,0.24537
best_epoch,6
best_val_loss,2.20947
epoch,6
loss,2.13424


Training 64 cells, fold 1...
Epoch 1/50
3334/3334 [==============================] - 54s 16ms/step - loss: 2.3232 - accuracy: 0.2133 - val_loss: 2.2532 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 52s 15ms/step - loss: 2.1895 - accuracy: 0.2167 - val_loss: 2.2319 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 52s 16ms/step - loss: 2.1715 - accuracy: 0.2276 - val_loss: 2.2359 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 54s 16ms/step - loss: 2.1596 - accuracy: 0.2320 - val_loss: 2.2168 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 52s 16ms/step - loss: 2.1446 - accuracy: 0.2425 - val_loss: 2.2194 - val_accuracy: 0.1813 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 52s 16ms/step - loss: 2.1255 - accuracy: 0.2551 - val_loss: 2.2093 - val_accuracy: 0.1834 - lr: 1.00

accuracy,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▆▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▅
lr,██████████████████▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▂▃▂▄▅▅▆▆▆▆▇█████████▇▇▇▇▇▆▆▆▆▆▆▆▆▇▇
val_loss,█▇█▇▇▆▅▄▄▅▃▃▃▂▂▁▁▂▁▁▁▁▁▁▁▂▁▂▂▃▃▃▄▄▅▅▆▆▇▇
accuracy,0.39961
best_epoch,26
best_val_loss,2.0077
epoch,49
loss,2.08634


Training 64 cells, fold 2...
Epoch 1/50
3334/3334 [==============================] - 53s 16ms/step - loss: 2.3437 - accuracy: 0.1957 - val_loss: 2.1500 - val_accuracy: 0.2502 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 52s 15ms/step - loss: 2.1867 - accuracy: 0.2142 - val_loss: 2.1379 - val_accuracy: 0.2504 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 54s 16ms/step - loss: 2.1726 - accuracy: 0.2230 - val_loss: 2.1308 - val_accuracy: 0.2509 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 54s 16ms/step - loss: 2.1621 - accuracy: 0.2275 - val_loss: 2.1285 - val_accuracy: 0.2546 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 52s 15ms/step - loss: 2.1553 - accuracy: 0.2345 - val_loss: 2.1221 - val_accuracy: 0.2553 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 52s 15ms/step - loss: 2.1415 - accuracy: 0.2397 - val_loss: 2.1161 - val_accuracy: 0.2575 - lr: 1.00

accuracy,▁▂▂▂▂▃▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇██████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▄▄▄▅
lr,████████████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▂▂▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇██████████████████▇
val_loss,▆▆▆▆▅▅▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▃▅▆▇█
accuracy,0.41362
best_epoch,37
best_val_loss,1.96919
epoch,49
loss,2.09426


...

accuracy,▁▁▁▂▂▃▃▄▄▅▅▅▆▆▆▇▇▇▇█████████▇▆▃▄████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▂▂▃▆▄▂▂▂▂▂▃▃▅
lr,████████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▂▂▂▁▄▅▄▅▆▅▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇███▇█▆▆▆▆▆▆▆▇
val_loss,██▇█▇▇▆▅▄▅▃▃▃▃▃▃▂▄▃▃▃▃▃▃▃▂▂▁▁▂▂▁▄▄▄▄▄▅▅▇
accuracy,0.44802
best_epoch,38
best_val_loss,1.94538
epoch,49
loss,1.98408


Training 256 cells, fold 2...
Epoch 1/50
3334/3334 [==============================] - 80s 24ms/step - loss: 2.2023 - accuracy: 0.2151 - val_loss: 2.1365 - val_accuracy: 0.2530 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.1590 - accuracy: 0.2266 - val_loss: 2.1383 - val_accuracy: 0.2523 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.1491 - accuracy: 0.2352 - val_loss: 2.1289 - val_accuracy: 0.2537 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.1396 - accuracy: 0.2434 - val_loss: 2.1235 - val_accuracy: 0.2542 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.1286 - accuracy: 0.2476 - val_loss: 2.1198 - val_accuracy: 0.2565 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.1159 - accuracy: 0.2518 - val_loss: 2.1072 - val_accuracy: 0.2577 - lr: 1.0

accuracy,▁▁▂▂▂▃▃▄▅▅▅▅▆▆▆▇▇▇▇███████████▇▇▇▇▇▇▇▇▇▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▇▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▃▅▆▆▇▇▇██
lr,████████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▂▃▄▅▆▆▆▇▇▇███████████████▇▇▇▇▇▇▇▇▇▇
val_loss,▄▄▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▂▄▅▆▆▇▇██
accuracy,0.42389
best_epoch,35
best_val_loss,1.97287
epoch,49
loss,2.22566


Training 256 cells, fold 3...
Epoch 1/50
3334/3334 [==============================] - 80s 24ms/step - loss: 2.1406 - accuracy: 0.2425 - val_loss: 2.3914 - val_accuracy: 0.1087 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 79s 24ms/step - loss: 2.0866 - accuracy: 0.2566 - val_loss: 2.3924 - val_accuracy: 0.1094 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 78s 23ms/step - loss: 2.0831 - accuracy: 0.2658 - val_loss: 2.3801 - val_accuracy: 0.1425 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 88s 26ms/step - loss: 2.0761 - accuracy: 0.2727 - val_loss: 2.3558 - val_accuracy: 0.1697 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 79s 24ms/step - loss: 2.0674 - accuracy: 0.2776 - val_loss: 2.3716 - val_accuracy: 0.1823 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 78s 24ms/step - loss: 2.0606 - accuracy: 0.2821 - val_loss: 2.3665 - val_accuracy: 0.1780 - lr: 1.0

accuracy,▁▂▂▂▃▃▃▄▅▆▆▆▇▇▇▇▇█▇██████████████▇▇▇████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▆▆▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▄▄▆▇▇█
lr,████████████▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▂▃▄▄▅▄▆▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇█████████
val_loss,███▇▇▇▆▅▃▂▂▃▂▂▁▂▁▂▂▁▂▁▂▂▂▂▂▁▁▁▁▁▁▂▂▃▄▄▅▆
accuracy,0.3942
best_epoch,38
best_val_loss,2.10453
epoch,49
loss,2.16057
